Importando as bibliotecas iniciais

In [ ]:
import netsquid as ns
import matplotlib.pyplot as plt
from Default_Setup.setups import networkSetup
from netsquid.examples.teleportation import example_sim_setup

Criando a classe do simulador

In [ ]:
class Simulation:
    def __init__(self, node_distance: float = 4e-3, depolarization_rate: float = 0, dephasing_rate: float = 0) -> None:
        # Criando a rede
        self.network = networkSetup(node_distance=node_distance, depolarization_rate=depolarization_rate, 
                               dephasing_rate=dephasing_rate)
        # Criando os nós de Alice e Bob
        self.node_a = self.network.get_node("Alice")
        self.node_b = self.network.get_node("Bob")
        # Gerando os protocolos de Alice e Bob e criando um coletor de fidelidade
        self.protocol_alice, self.protocol_bob, self.dc = example_sim_setup(self.node_a, self.node_b)
        self.protocol_alice.start()
        self.protocol_bob.start()

    def creatQuantumConnection(self, label: str = "quantum") -> None:
        # Criando um canal quântico
        self.quantum_connection = self.network.get_connection(self.node_a, self.node_b, label=label)
        self.cycle_runtime = (self.quantum_connection.subcomponents["qsource"].subcomponents["internal_clock"]
                 .models["timing_model"].delay)
        
    def runSimulation(self, cycle_times: int = 100) -> None:
        # Rodando o simulador
        ns.sim_run(self.cycle_runtime * cycle_times)
        self.fidelity = self.dc.dataframe['fidelity'].mean()
        print(f"A fidelidade do protocolo de teletrasporte foi de: {self.fidelity:.3f}")

    def returnFidelity(self) -> float:
        return self.fidelity


print(f"This example module is located at: {ns.examples.teleportation.__file__}")

Criando um gerador de gráfico

In [ ]:
class Graphic:
    def __init__(self, simulation: Simulation, data_file_name: str = 'fidelitydata.txt') -> None:
        self.fidelity_axis = []
        self.data_count = 0
        self.y_label = 'Fidelity'
        self.x_label = 'Execution number'
        self.file_name = data_file_name
        self.fidelity = simulation.returnFidelity()


    def testFile(self, create_file: bool=True) -> bool:
        try:
            with open(self.file_name, 'r') as file:
                return True
            
        except:
            if create_file:
                with open(self.file_name, 'w+'):
                    print(f'Nenhum arquivo encontrado. Criado o arquivo "{self.file_name}"')
            return False
    

    def deleteData(self) -> None:
        test = self.testFile(False)
        if test:
            with open(self.file_name, 'w') as file:
                print("Dados apagados com sucesso")
                return
            
        print("Sem dados para apagar")

    def readDataCollector(self) -> None:
        test = self.testFile()
        if test:
            with open(self.file_name, 'r') as file:
                temp_list = []
                temp_count = 0
                for line in file:
                    if line != None:
                        temp_list.append(float(line.replace('\n', '')))
                        temp_count += 1
                self.fidelity_axis = temp_list
                self.data_count = temp_count
                return
        
        print('Nenhum dado existente')
        return
    
        
    def addOnDataCollector(self) -> None:
        self.testFile()
        with open(self.file_name, '+a') as file:
            file.write(f'{self.fidelity}\n')
            
    
    def printData(self) -> None:
        test = self.testFile()
        if test:
            print("Os dados são: ")
            with open(self.file_name, 'r') as file:
                for line in file:
                    print(line)
                return
        
        print("Nenhum dado pôde ser imprimido")
        return


    def generateGraph(self) -> None:
        plt.plot(self.fidelity_axis, scalex=self.data_count)
        plt.ylabel(self.y_label)
        plt.xlabel(self.x_label)
        plt.show()




Parâmetros do simulador

In [ ]:
simulator = Simulation(4e-3, 1e7, 0.2)
simulator.creatQuantumConnection()
simulator.runSimulation()
graphic = Graphic(simulation=simulator)
graphic.addOnDataCollector()
resp = input("Você gostaria de observar o gráfico dos dados? [s/n] ").split()
if resp[0] in 'Ss':
    graphic.readDataCollector()
    graphic.generateGraph()
    graphic.printData()
    resp = input("Deseja apagar todos os dados salvos? [s/n] ").split()
    if resp[0] in 'sS':
        graphic.deleteData()

else:
    print("Simulação finalizada com sucesso")
